In [1]:
import pandas as pd

In [20]:
attri_allo = pd.read_csv('attribution_allocation_student_data.csv')
channel_sp = pd.read_csv('channel_spend_student_data.csv', names=['channel','spend'], skiprows=[0])

In [21]:
attri_allo

,convert_TF,touch1,touch2,touch3,touch4,touch5,tier
0,True,email,referral,referral,NaN,NaN,1
1,True,referral,referral,email,NaN,NaN,1
2,True,referral,NaN,NaN,NaN,NaN,1
3,True,email,NaN,NaN,NaN,NaN,1
4,True,referral,referral,social,referral,NaN,1
...,...,...,...,...,...,...,...
68119,False,direct,NaN,NaN,NaN,NaN,3
68120,False,direct,email,referral,NaN,NaN,3
68121,False,direct,referral,NaN,NaN,NaN,3
68122,False,email,direct,NaN,NaN,NaN,3


In [22]:
channel_sp

,channel,spend
0,tier1,"{'email': 1000.0, 'social': 1000.0000000000001..."
1,tier2,"{'email': 2000.0, 'social': 2000.0, 'display':..."
2,tier3,"{'email': 3000.0, 'social': 3000.0, 'display':..."


# Part 1 Attribution

## 1.1 Test 3 methods for allocation

In [23]:
for i in [1, 2, 3]:
    print("The conversion rate for tier %d is %.2f%%." % (i, 100*attri_allo.groupby(by='tier').mean().loc[i, 'convert_TF']))

The conversion rate for tier 1 is 24.87%.
The conversion rate for tier 2 is 21.54%.
The conversion rate for tier 3 is 17.80%.


## 1.2 Calculate average CAC for each of the channels

In [24]:
# extract spend allocation information
tier1 = eval(channel_sp.loc[0, 'spend'])
tier2 = eval(channel_sp.loc[1, 'spend'])
tier3 = eval(channel_sp.loc[2, 'spend'])
for i in [0, 1, 2]:
    print(channel_sp.loc[i, 'spend'])

{'email': 1000.0, 'social': 1000.0000000000001, 'display': 1000.0, 'paid_search': 999.9999999999999, 'referral': 1000.0, 'organic_search': 0.0, 'direct': 0.0}
{'email': 2000.0, 'social': 2000.0, 'display': 2000.0, 'paid_search': 2000.0, 'referral': 2000.0, 'organic_search': 0.0, 'direct': 0.0}
{'email': 3000.0, 'social': 3000.0, 'display': 3000.0, 'paid_search': 3000.0, 'referral': 3000.0, 'organic_search': 0.0, 'direct': 0.0}


In [25]:
# calculate the number of customers converted through each way of allocation
converted = attri_allo.loc[attri_allo['convert_TF']==True]
converted_1 = converted.loc[converted['tier']==1]
converted_2 = converted.loc[converted['tier']==2]
converted_3 = converted.loc[converted['tier']==3]
for i in [1, 2, 3]:
    print("The number of customers converted through tier %d is %d." % (i, converted['tier'].value_counts()[i]))

The number of customers converted through tier 1 is 2824.
The number of customers converted through tier 2 is 4892.
The number of customers converted through tier 3 is 6063.


### Last interaction

In [26]:
touch_1 = converted_1.loc[:,'touch1':'touch5'].reset_index()
touch_1.drop(['index'], axis=1, inplace=True)
touch_2 = converted_2.loc[:,'touch1':'touch5'].reset_index()
touch_2.drop(['index'], axis=1, inplace=True)
touch_3 = converted_3.loc[:,'touch1':'touch5'].reset_index()
touch_3.drop(['index'], axis=1, inplace=True)

In [27]:
temp1 = touch_1.stack().reset_index()
temp2 = touch_2.stack().reset_index()
temp3 = touch_3.stack().reset_index()

last_touch_1 = pd.DataFrame(touch_1.stack().reset_index().groupby('level_0')['level_1'].max()).reset_index()
last_touch_2 = pd.DataFrame(touch_2.stack().reset_index().groupby('level_0')['level_1'].max()).reset_index()
last_touch_3 = pd.DataFrame(touch_3.stack().reset_index().groupby('level_0')['level_1'].max()).reset_index()

In [29]:
last_touch_1 = last_touch_1.merge(temp1, on=['level_0','level_1'], how='left')
last_touch_2 = last_touch_2.merge(temp2, on=['level_0','level_1'], how='left')
last_touch_3 = last_touch_3.merge(temp3, on=['level_0','level_1'], how='left')

In [31]:
last_convert_1 = pd.DataFrame(last_touch_1[0].value_counts())
last_convert_2 = pd.DataFrame(last_touch_2[0].value_counts())
last_convert_3 = pd.DataFrame(last_touch_3[0].value_counts())

In [34]:
last_convert_3

,0
referral,2670
social,1256
display,959
email,686
paid_search,484
organic_search,5
direct,3


In [51]:
for channel in list(tier1.keys()):
    try:
        CAC1 = tier1[channel]/last_convert_1.loc[channel,0]
        #print(CAC1)
        CAC2 = tier2[channel]/last_convert_2.loc[channel,0]
        #print(CAC2)
        CAC3 = tier3[channel]/last_convert_3.loc[channel,0]
        #print(CAC3)
        Avg_CAC = round((CAC1+CAC2+CAC3)/3,4)
    except:
        Avg_CAC = 'NA'
    print('By using Last Interaction method, the average CAC for {} is {}.'.format(channel, Avg_CAC))

By using Last Interaction method, the average CAC for email is 3.8257.
By using Last Interaction method, the average CAC for social is 1.9557.
By using Last Interaction method, the average CAC for display is 2.4796.
By using Last Interaction method, the average CAC for paid_search is 5.8788.
By using Last Interaction method, the average CAC for referral is 0.9692.
By using Last Interaction method, the average CAC for organic_search is 0.0.
By using Last Interaction method, the average CAC for direct is NA.


### First interaction

In [53]:
first_convert_1 = pd.DataFrame(touch_1['touch1'].value_counts())
first_convert_2 = pd.DataFrame(touch_2['touch1'].value_counts())
first_convert_3 = pd.DataFrame(touch_3['touch1'].value_counts())

In [54]:
first_convert_1

,touch1
referral,1493
display,484
social,471
email,246
paid_search,128
direct,2


In [55]:
for channel in list(tier1.keys()):
    try:
        CAC1 = tier1[channel]/first_convert_1.loc[channel,'touch1']
        #print(CAC1)
        CAC2 = tier2[channel]/first_convert_2.loc[channel,'touch1']
        #print(CAC2)
        CAC3 = tier3[channel]/first_convert_3.loc[channel,'touch1']
        #print(CAC3)
        Avg_CAC = round((CAC1+CAC2+CAC3)/3, 4)
    except:
        Avg_CAC = 'NA'
    print('By using First Interaction method, the average CAC for {} is {}.'.format(channel, Avg_CAC))

By using First Interaction method, the average CAC for email is 4.6032.
By using First Interaction method, the average CAC for social is 2.5553.
By using First Interaction method, the average CAC for display is 2.7862.
By using First Interaction method, the average CAC for paid_search is 8.4088.
By using First Interaction method, the average CAC for referral is 0.7744.
By using First Interaction method, the average CAC for organic_search is NA.
By using First Interaction method, the average CAC for direct is 0.0.


### Linear

In [121]:
linear_1 = touch_1.stack().reset_index()
sum_1 = touch_1.stack().reset_index().groupby('level_0').count().reset_index()
linear_1 = linear_1.merge(sum_1, on=['level_0'], how='outer')
linear_1.drop(['level_0','level_1_x','level_1_y'], axis = 1, inplace = True)
linear_1.columns = ['channel', 'touch_num']
linear_1['touch_num'] = 1/linear_1['touch_num']
linear_convert_1 = linear_1.groupby('channel').sum()
linear_convert_1

,touch_num
channel,
direct,2.250000
display,466.433333
email,243.666667
organic_search,2.350000
paid_search,127.166667
referral,1503.116667
social,479.016667


In [122]:
linear_2 = touch_2.stack().reset_index()
sum_2 = touch_2.stack().reset_index().groupby('level_0').count().reset_index()
linear_2 = linear_2.merge(sum_2, on=['level_0'], how='outer')
linear_2.drop(['level_0','level_1_x','level_1_y'], axis = 1, inplace = True)
linear_2.columns = ['channel', 'touch_num']
linear_2['touch_num'] = 1/linear_2['touch_num']
linear_convert_2 = linear_2.groupby('channel').sum()
linear_convert_2

,touch_num
channel,
direct,1.333333
display,715.183333
email,419.050000
organic_search,3.250000
paid_search,246.633333
referral,2682.166667
social,824.383333


In [123]:
linear_3 = touch_3.stack().reset_index()
sum_3 = touch_3.stack().reset_index().groupby('level_0').count().reset_index()
linear_3 = linear_3.merge(sum_3, on=['level_0'], how='outer')
linear_3.drop(['level_0','level_1_x','level_1_y'], axis = 1, inplace = True)
linear_3.columns = ['channel', 'touch_num']
linear_3['touch_num'] = 1/linear_3['touch_num']
linear_convert_3 = linear_3.groupby('channel').sum()
linear_convert_3

,touch_num
channel,
direct,5.366667
display,849.716667
email,595.133333
organic_search,5.233333
paid_search,334.183333
referral,3285.283333
social,988.083333


In [59]:
for channel in list(tier1.keys()):
    try:
        CAC1 = tier1[channel]/linear_convert_1.loc[channel,'touch_num']
        #print(CAC1)
        CAC2 = tier2[channel]/linear_convert_2.loc[channel,'touch_num']
        #print(CAC2)
        CAC3 = tier3[channel]/linear_convert_3.loc[channel,'touch_num']
        #print(CAC3)
        Avg_CAC = round((CAC1+CAC2+CAC3)/3,4)
    except:
        Avg_CAC = 'NA'
    print('By using Linear method, the average CAC for {} is {}.'.format(channel, Avg_CAC))

By using Linear method, the average CAC for email is 4.6392.
By using Linear method, the average CAC for social is 2.5166.
By using Linear method, the average CAC for display is 2.8237.
By using Linear method, the average CAC for paid_search is 8.3167.
By using Linear method, the average CAC for referral is 0.7747.
By using Linear method, the average CAC for organic_search is 0.0.
By using Linear method, the average CAC for direct is 0.0.


# Part 2 Allocation

## 2.1 calculate the marginal CAC by spending tier by channel

### Last Interaction

In [116]:
last_convert = last_convert_1.reset_index().merge(last_convert_2.reset_index(), on=['index'], how='outer')
last_convert = last_convert.merge(last_convert_3.reset_index(), on=['index'], how='outer')
last_convert.columns = ['channel', 'tier1_num', 'tier2_num', 'tier3_num']
last_convert

,channel,tier1_num,tier2_num,tier3_num
0,referral,1187,2124.0,2670
1,social,622,1069.0,1256
2,display,531,824.0,959
3,email,299,532.0,686
4,paid_search,180,340.0,484
5,direct,3,NaN,3
6,organic_search,2,3.0,5


In [117]:
last_convert['marginal_num_1'] = last_convert['tier1_num']
last_convert['marginal_num_2'] = last_convert['tier2_num'] - last_convert['tier1_num']
last_convert['marginal_num_3'] = last_convert['tier3_num'] - last_convert['tier2_num']
last_convert

,channel,tier1_num,tier2_num,tier3_num,marginal_num_1,marginal_num_2,marginal_num_3
0,referral,1187,2124.0,2670,1187,937.0,546.0
1,social,622,1069.0,1256,622,447.0,187.0
2,display,531,824.0,959,531,293.0,135.0
3,email,299,532.0,686,299,233.0,154.0
4,paid_search,180,340.0,484,180,160.0,144.0
5,direct,3,NaN,3,3,NaN,NaN
6,organic_search,2,3.0,5,2,1.0,2.0


In [118]:
last_convert['marginal_cost_1'] = 0
last_convert['marginal_cost_2'] = 0
last_convert['marginal_cost_3'] = 0
for i in range(0,len(last_convert)):
    channel = last_convert.loc[i, 'channel']
    last_convert.loc[i, 'marginal_cost_1'] = tier1[channel]
    last_convert.loc[i, 'marginal_cost_2'] = tier2[channel] - tier1[channel]
    last_convert.loc[i, 'marginal_cost_3'] = tier3[channel] - tier2[channel]
last_convert

,channel,tier1_num,tier2_num,tier3_num,marginal_num_1,marginal_num_2,marginal_num_3,marginal_cost_1,marginal_cost_2,marginal_cost_3
0,referral,1187,2124.0,2670,1187,937.0,546.0,1000.0,1000.0,1000.0
1,social,622,1069.0,1256,622,447.0,187.0,1000.0,1000.0,1000.0
2,display,531,824.0,959,531,293.0,135.0,1000.0,1000.0,1000.0
3,email,299,532.0,686,299,233.0,154.0,1000.0,1000.0,1000.0
4,paid_search,180,340.0,484,180,160.0,144.0,1000.0,1000.0,1000.0
5,direct,3,NaN,3,3,NaN,NaN,0.0,0.0,0.0
6,organic_search,2,3.0,5,2,1.0,2.0,0.0,0.0,0.0


In [119]:
last_convert['marginal_CAC_1'] = last_convert['marginal_cost_1']/last_convert['marginal_num_1']
last_convert['marginal_CAC_2'] = last_convert['marginal_cost_2']/last_convert['marginal_num_2']
last_convert['marginal_CAC_3'] = last_convert['marginal_cost_3']/last_convert['marginal_num_3']

In [120]:
marginal_CAC = last_convert.loc[:, ['channel','marginal_CAC_1','marginal_CAC_2','marginal_CAC_3']]
marginal_CAC

,channel,marginal_CAC_1,marginal_CAC_2,marginal_CAC_3
0,referral,0.842460,1.067236,1.831502
1,social,1.607717,2.237136,5.347594
2,display,1.883239,3.412969,7.407407
3,email,3.344482,4.291845,6.493506
4,paid_search,5.555556,6.250000,6.944444
5,direct,0.000000,NaN,NaN
6,organic_search,0.000000,0.000000,0.000000


3,000 for referral, 3,000 for social, 2,000 for display, 2,000 for email, 1,000 for paid_search.

### Linear

In [127]:
linear_convert = linear_convert_1.reset_index().merge(linear_convert_2.reset_index(), on=['channel'], how='outer')
linear_convert = linear_convert.merge(linear_convert_3.reset_index(), on=['channel'], how='outer')
linear_convert.columns = ['channel', 'tier1_num', 'tier2_num', 'tier3_num']
linear_convert

,channel,tier1_num,tier2_num,tier3_num
0,direct,2.250000,1.333333,5.366667
1,display,466.433333,715.183333,849.716667
2,email,243.666667,419.050000,595.133333
3,organic_search,2.350000,3.250000,5.233333
4,paid_search,127.166667,246.633333,334.183333
5,referral,1503.116667,2682.166667,3285.283333
6,social,479.016667,824.383333,988.083333


In [128]:
linear_convert['marginal_num_1'] = linear_convert['tier1_num']
linear_convert['marginal_num_2'] = linear_convert['tier2_num'] - linear_convert['tier1_num']
linear_convert['marginal_num_3'] = linear_convert['tier3_num'] - linear_convert['tier2_num']
linear_convert

,channel,tier1_num,tier2_num,tier3_num,marginal_num_1,marginal_num_2,marginal_num_3
0,direct,2.250000,1.333333,5.366667,2.250000,-0.916667,4.033333
1,display,466.433333,715.183333,849.716667,466.433333,248.750000,134.533333
2,email,243.666667,419.050000,595.133333,243.666667,175.383333,176.083333
3,organic_search,2.350000,3.250000,5.233333,2.350000,0.900000,1.983333
4,paid_search,127.166667,246.633333,334.183333,127.166667,119.466667,87.550000
5,referral,1503.116667,2682.166667,3285.283333,1503.116667,1179.050000,603.116667
6,social,479.016667,824.383333,988.083333,479.016667,345.366667,163.700000


In [129]:
linear_convert['marginal_cost_1'] = 0
linear_convert['marginal_cost_2'] = 0
linear_convert['marginal_cost_3'] = 0
for i in range(0,len(linear_convert)):
    channel = linear_convert.loc[i, 'channel']
    linear_convert.loc[i, 'marginal_cost_1'] = tier1[channel]
    linear_convert.loc[i, 'marginal_cost_2'] = tier2[channel] - tier1[channel]
    linear_convert.loc[i, 'marginal_cost_3'] = tier3[channel] - tier2[channel]
linear_convert

,channel,tier1_num,tier2_num,tier3_num,marginal_num_1,marginal_num_2,marginal_num_3,marginal_cost_1,marginal_cost_2,marginal_cost_3
0,direct,2.250000,1.333333,5.366667,2.250000,-0.916667,4.033333,0.0,0.0,0.0
1,display,466.433333,715.183333,849.716667,466.433333,248.750000,134.533333,1000.0,1000.0,1000.0
2,email,243.666667,419.050000,595.133333,243.666667,175.383333,176.083333,1000.0,1000.0,1000.0
3,organic_search,2.350000,3.250000,5.233333,2.350000,0.900000,1.983333,0.0,0.0,0.0
4,paid_search,127.166667,246.633333,334.183333,127.166667,119.466667,87.550000,1000.0,1000.0,1000.0
5,referral,1503.116667,2682.166667,3285.283333,1503.116667,1179.050000,603.116667,1000.0,1000.0,1000.0
6,social,479.016667,824.383333,988.083333,479.016667,345.366667,163.700000,1000.0,1000.0,1000.0


In [130]:
linear_convert['marginal_CAC_1'] = linear_convert['marginal_cost_1']/linear_convert['marginal_num_1']
linear_convert['marginal_CAC_2'] = linear_convert['marginal_cost_2']/linear_convert['marginal_num_2']
linear_convert['marginal_CAC_3'] = linear_convert['marginal_cost_3']/linear_convert['marginal_num_3']

In [131]:
linear_marginal_CAC = linear_convert.loc[:, ['channel','marginal_CAC_1','marginal_CAC_2','marginal_CAC_3']]
linear_marginal_CAC

,channel,marginal_CAC_1,marginal_CAC_2,marginal_CAC_3
0,direct,0.000000,-0.000000,0.000000
1,display,2.143929,4.020101,7.433102
2,email,4.103967,5.701796,5.679129
3,organic_search,0.000000,0.000000,0.000000
4,paid_search,7.863696,8.370536,11.422045
5,referral,0.665284,0.848140,1.658054
6,social,2.087610,2.895473,6.108735


3,000 for referral, 3,000 for social, 2,000 for display, 3,000 for email.